In [ ]:
import subprocess
import pandas as pd
import json
import uuid

In [ ]:
#define variables
recipes = pd.read_excel(r'') #Enter the path to your 01_Input_Recipes File
ingredients = pd.read_excel(r'') #Enter the path to your 02_Input_LU_Ingredient File
sorter_file = pd.read_excel(r'') #Enter the path to your 03_Input_LU_Sort File
bearer = '' # Enter your bearer here
project_id = '' # Enter your project_id here

In [ ]:
# Define functions
def format_amount(value):
    if value % 1 == 0:  
        return str(int(value))
    else:
        return f"{value:.1f}" 

In [ ]:
#Get Shopping list items
recipes_chosen = recipes.dropna(subset=['Portions'])
ingredients_chosen = pd.merge(ingredients,recipes_chosen,on=['ID'])
ingredients_chosen['Adjusted_Amount'] = ingredients_chosen['Amount'] * ingredients_chosen['Portions'] / ingredients_chosen['For_portions']
shoppinglist = ingredients_chosen.groupby(['Ingredient', 'Unit'])['Adjusted_Amount'].sum().reset_index()

# Apply format_amount to Adjusted_Amount
shoppinglist['Adjusted_Amount'] = shoppinglist['Adjusted_Amount'].apply(format_amount)

# Concatenate 'Ingredient', 'Adjusted_Amount' und 'Unit'
shoppinglist['Content'] = shoppinglist['Ingredient'] + ' - ' + shoppinglist['Adjusted_Amount'] + ' ' + shoppinglist['Unit']

# Sort in the correct order
sorted_shoppinglist = pd.merge(shoppinglist,sorter_file,on=['Ingredient'])
sorted_shoppinglist = sorted_shoppinglist.sort_values(by=['sort_number', 'Ingredient'], ascending=[True, True])

# Show only column 'Content'
sorted_shoppinglist = sorted_shoppinglist[['Content']]

In [ ]:
get_sync_call = r'curl https://api.todoist.com/sync/v9/sync -H "Authorization: Bearer '+bearer+r'" -d "sync_token=*" -d "resource_types=[\"items\", \"projects\"]"'
sync_response = subprocess.run(get_sync_call, capture_output=True, text=True, shell=True)
data = json.loads(sync_response.stdout)
sync_token = data.get("sync_token", None)

In [ ]:
for i,r in sorted_shoppinglist.iterrows():
    temp_id = str(uuid.uuid4())
    uu_id = str(uuid.uuid4())
    put_item_call = r'curl https://api.todoist.com/sync/v9/sync ^ -H "Authorization: Bearer '+bearer+r'"  -d "sync_token='+sync_token+r'"  -d "resource_types=[\"projects\", \"items\"]"  -d "commands=[{\"type\": \"item_add\", \"temp_id\": \"'+temp_id+r'\", \"uuid\": \"'+uu_id+r'\", \"args\": {\"project_id\": \"'+project_id+r'\", \"content\": \"'+r['Content']+r'\"}}]"'
    subprocess.run(put_item_call, capture_output=False, text=True, shell=True)
    print(r['Content'])